In [11]:
***REMOVED***
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator



2024-01-30 18:06:07.863553: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 18:06:07.885935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 18:06:07.885972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 18:06:07.886759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 18:06:07.890998: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 18:06:07.894806: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [12]:
df = pd.read_csv("../tags_processed_stages/dafre_tags.csv")


In [13]:
len(df)

463437

In [14]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42) # 80% training, 20% testing
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255, # Rescale the image by normalizing it.
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [16]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../fullMin256/', # Replace with the path to the directory where images are stored
    x_col='dir', # Column in dataframe that contains the filenames
    y_col='class_id', # Column in dataframe that has the target labels
    target_size=(150, 150), # Size to which all images will be resized
    batch_size=32,
    class_mode='raw'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='class_id',
    target_size=(150, 150),
    batch_size=32,
    class_mode='raw'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='class_id',
    target_size=(150, 150),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

Found 0 validated image filenames.


/home/leogu/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 278061 invalid image filename(s) in x_col="dir". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames.


/home/leogu/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 92688 invalid image filename(s) in x_col="dir". These filename(s) will be ignored.
  warnings.warn(


Found 0 validated image filenames.
